<a href="https://colab.research.google.com/github/riztikamerista/2025_PBO_TI1C/blob/main/tubes_pbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import csv
import folium
import os
from streamlit_folium import st_folium
from folium.plugins import Fullscreen

# --- Setup Path ---
CSV_PATH = "data/lokasi_budaya.csv"
os.makedirs("data", exist_ok=True)

# --- Setup Page ---
st.set_page_config(page_title="Pemetaan Situs Budaya", layout="wide")
st.title("📍 Pemetaan Situs Sejarah dan Budaya Lokal")

# --- Session Init ---
if "nama" not in st.session_state:
    st.session_state.nama = ""
    st.session_state.latitude = ""
    st.session_state.longitude = ""
    st.session_state.deskripsi = ""
    st.session_state.agama = ""

# --- Form Tambah Data ---
with st.form("form_situs"):
    st.subheader("➕ Tambah Situs Budaya Baru")
    nama = st.text_input("Nama Tempat", value=st.session_state.nama, key="nama")
    latitude = st.text_input("Latitude", value=st.session_state.latitude, key="latitude")
    longitude = st.text_input("Longitude", value=st.session_state.longitude, key="longitude")
    tipe = st.selectbox("Tipe", ["Museum", "Cagar Budaya", "Tempat Ibadah"])
    deskripsi = st.text_area("Deskripsi", value=st.session_state.deskripsi, key="deskripsi")
    agama = st.text_input("Agama (jika tempat ibadah)", value=st.session_state.agama, key="agama")

    submitted = st.form_submit_button("✅ Tambahkan")

    if submitted:
        try:
            lat = float(latitude)
            lon = float(longitude)

            file_exists = os.path.exists(CSV_PATH)
            with open(CSV_PATH, 'a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                if not file_exists or os.stat(CSV_PATH).st_size == 0:
                    writer.writerow(["Nama", "Latitude", "Longitude", "Tipe", "Deskripsi", "Agama"])
                writer.writerow([nama, lat, lon, tipe, deskripsi, agama])

            st.success("✅ Data berhasil ditambahkan!")

            for field in ["nama", "latitude", "longitude", "deskripsi", "agama"]:
                del st.session_state[field]
            st.rerun()

        except ValueError:
            st.error("⚠️ Latitude dan Longitude harus berupa angka!")

# --- Baca Data ---
try:
    df = pd.read_csv(CSV_PATH, encoding='utf-8')
    df = df.dropna(subset=["Latitude", "Longitude"])
    df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
    df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")
    df = df.dropna(subset=["Latitude", "Longitude"])
except:
    st.warning("⚠️ Belum ada data. Tambahkan dulu.")
    st.stop()

if df.empty:
    st.warning("⚠️ CSV kosong.")
    st.stop()

# --- Pilih Lokasi ---
st.subheader("🎯 Pilih Situs untuk Diperbesar di Peta")
pilihan = st.selectbox("Pilih situs:", df["Nama"].unique())

baris = df[df["Nama"] == pilihan].iloc[0]
lat_focus = float(baris["Latitude"])
lon_focus = float(baris["Longitude"])

# --- Layout Dua Kolom ---
col1, col2 = st.columns([2, 1])

with col1:
    st.subheader("🗺️ Peta Interaktif")
    m = folium.Map(location=[lat_focus, lon_focus], zoom_start=17, tiles="OpenStreetMap")
    Fullscreen().add_to(m)

    for _, row in df.iterrows():
        try:
            lat = float(row["Latitude"])
            lon = float(row["Longitude"])
            popup = f"<b>{row['Nama']}</b><br><i>{row['Tipe']}</i><br>{row['Deskripsi']}"
            folium.Marker(location=[lat, lon], popup=popup, tooltip=row["Nama"]).add_to(m)
        except:
            continue

    st_folium(m, height=600)

with col2:
    st.subheader("📋 Detail Lokasi Terpilih")
    st.markdown(f"**🏷️ Nama:** {baris['Nama']}")
    st.markdown(f"**📌 Koordinat:** {baris['Latitude']}, {baris['Longitude']}")
    st.markdown(f"**🏛️ Tipe:** {baris['Tipe']}")
    st.markdown(f"**📖 Deskripsi:** {baris['Deskripsi']}")
    if str(baris['Agama']).strip():
        st.markdown(f"**🕉️ Agama:** {baris['Agama']}")